In [2]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
from mlxtend.preprocessing import minmax_scaling

In [3]:
response = requests.get("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")


<Response [200]>

In [4]:
# Criando o dataframe:
response = response.content.decode("utf-8")
result = [x.split(',') for x in response.split('\n')]
df = pd.DataFrame(result[1:], columns= result[0])

In [5]:
# Substituindo os valores vazios por 0:
df = df.replace("", 0, regex=True)

In [6]:
#  Removendo colunas com valores constantes:
df.drop(["country", "city"], axis=1, inplace=True)
df = df[:-1]

In [7]:
# Transformando a coluna de data para o formato apropriado:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [8]:
# Como a maioria das colunas está em um formato numérico, vamos converter elas para float:
for column in df.columns:
    try:
        df[column] = df[column].astype(float)
    except:
        continue

In [9]:
# Colocando as semanas epidemiológicas na ordem direta:
for row in df.index:
    if df["epi_week"][row] > 200:
        df["epi_week"][row] = (df["epi_week"][row]-200+105)
    elif df["epi_week"][row] > 100:
        df["epi_week"][row] = (df["epi_week"][row]-100+53)
        
# Acabei nem usando isso aqui

C:\Users\Giga\AppData\Local\Temp\ipykernel_12656\725959677.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Giga\AppData\Local\Temp\ipykernel_12656\725959677.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# Separando os dados a nível nacional:
df_total = df[df["state"]=="TOTAL"]

In [11]:
# Criando um sub-dataframe para análise de variações de novos casos:
days = pd.DataFrame({
    "newDeaths": [row for row in df_total["newDeaths"]],
    "newCases" : [row for row in df_total["newCases"]],
    "date"     : [row for row in df_total["date"]],
    "suspects" : [row for row in df_total["suspects"]],
    "vaccinated" : [row for row in df_total["vaccinated"]]
    
})

In [12]:
"""
FUNÇÃO PARA FAZER A AGREGAÇÃO DE DADOS POR MÊS E POR SEMANA:
"""
months = days.set_index("date").resample('M').sum()
display(months.head())

weeks = days.set_index("date").resample('W').sum()

"""
Fazendo essa agregação não podemos trabalhar com colunas de valores agregados!!!! 
Por exemplo, a coluna 'deaths' soma diversas vezes a coluna 'newDeaths', 
e quando ela é agregada ela soma esses valores mais de uma vez!

Isso gera um número absurdo de casos, além de dados errados!

Recomendo usar as colunas newDeaths e newCases da base de dados original,
as outras tem que ser avaliadas antes de serem usadas!!
""";

,newDeaths,newCases,suspects,vaccinated
date,,,,
2020-02-29,0.0,2.0,0.0,0.0
2020-03-31,202.0,5822.0,398135.0,0.0
2020-04-30,5778.0,81297.0,3979439.0,0.0
2020-05-31,23385.0,428975.0,10400746.0,0.0
2020-06-30,30416.0,896462.0,19133899.0,0.0


In [13]:
"""
Ao invés de usar resample poderiamos pensar em usar groupby, mas isso dificultaria - muito - a plotagem.

Groupby com multy index:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month])['newDeaths'].sum()

Groupby com index resetado:
monthly_deaths = df_total.groupby([df_total.date.dt.year, df_total.date.dt.month], as_index=False)['newDeaths'].sum()


""";

In [14]:
# Reescalando os dados, facilitar a plotagem:
scaled_months = minmax_scaling(months, columns = ["newCases","newDeaths", "suspects","vaccinated"])
scaled_months.head()

,newCases,newDeaths,suspects,vaccinated
date,,,,
2020-02-29,0.000000,0.000000,0.000000,0.0
2020-03-31,0.001751,0.002453,0.001449,0.0
2020-04-30,0.024465,0.070152,0.014478,0.0
2020-05-31,0.129094,0.283923,0.037841,0.0
2020-06-30,0.269778,0.369288,0.069614,0.0


In [15]:
#cores nos plots
colors = px.colors.qualitative.swatches()

In [25]:
fig_infec_mes = px.line(months, x=months.index, y="newCases",
               markers= True,
               title='Infecções Divulgadas ao Mês',
               labels = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["royalblue"])
fig_infec_mes.update_layout(font_color='blue',
                title_font_color='red',
                title_font_size = 20,
                title_x = 0.5)
fig_infec_mes.show()

In [29]:
fig_mortes_mes = px.line(months, x=months.index, y="newDeaths",
               markers= True,
               title='Óbitos Divulgados ao Mês',
               labels = {
                "deathsMS": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["red"])
fig_mortes_mes.update_layout(font_color='red',
                title_font_color='orange',
                title_font_size = 20,
                title_x = 0.5)
fig_mortes_mes.show()

In [27]:
fig_susp_mes = px.line(months, x=months.index, y="suspects",
                markers= True,
                title='Casos Suspeitos Divulgados por Mês',
                labels  = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["orange"])
fig_susp_mes.update_layout(font_color='orange',
                title_font_color='red',
                title_font_size = 20,
                title_x = 0.5)
fig_susp_mes.show()

In [28]:
fig_vacinados_mes = px.line(months, x=months.index, y="vaccinated",
                markers= True,
                title='Quantidade de Vacinados por Mês',
                labels  = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["green"])
fig_vacinados_mes.update_layout(font_color='green',
                title_font_color='black',
                title_font_size = 20,
                title_x = 0.5)
fig_vacinados_mes.show()

In [32]:
# plotando novos casos, suspeitas, vacinados e óbitos
fig_infec_susp_vacin_mortes = go.Figure()
fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                               y = scaled_months["newCases"],
                               mode = 'lines+markers',
                               name = "Infecções",
                               line = dict(color='royalblue',width=2,dash='dot')))
fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                               y = scaled_months["suspects"],
                               mode = 'lines+markers',
                               name = "Suspeitos",
                               line = dict(color='orange',width=2,dash='dot')))
fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                               y = scaled_months["vaccinated"],
                               mode = 'lines+markers',
                               name = "Vacinados"))
fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                               y = scaled_months["newDeaths"],
                               mode = 'lines+markers',
                               name = "Óbitos",
                               line = dict(color='red')
                               ))
fig_infec_susp_vacin_mortes.update_layout(
                                title='Comparando Novos casos, suspeitos, vacinados e Óbitos Mês',
                                title_font_size = 20,
                                title_font_color = 'black',
                                title_x = 0.5)
fig_infec_susp_vacin_mortes.show()

In [33]:
# Plotando relaçao das infecções e óbitos por covid:
fig_infec_mortes_covid = go.Figure()
fig_infec_mortes_covid = fig_infec_mortes_covid.add_trace(go.Scatter(x = scaled_months.index,
                               y = scaled_months["newCases"], 
                               mode = 'lines+markers',
                               name = "Infecções",
                               line = dict(color='royalblue',width=2,dash='dot')))

fig_infec_mortes_covid = fig_infec_mortes_covid.add_trace(go.Bar(x = scaled_months.index,
                           y = scaled_months["newDeaths"], 
                           name = "Óbitos"))

fig_infec_mortes_covid.add_vline(x='2021-01-17',
                line_width=2,
                line_dash="dot")

fig_infec_mortes_covid.add_annotation(x="2021-01-17", y = 1,
            text="Início das Vacinações no Brasil",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor= "#FFFFFF")

fig_infec_mortes_covid.update_layout(
            title='Relação das Infecções e Óbitos por Covid',
            title_font_size = 20,
            title_font_color = 'black',
            title_x = 0.5)

fig_infec_mortes_covid.show()

In [35]:
# Analisando a variação de novos casos divulgados:
days_var = days.set_index("date").pct_change()
weeks_var = weeks.pct_change()
months_var = months.pct_change()

In [36]:
# Criando um dicionário para os Dfs
dfs = {"days" : days_var[2:], "weeks": weeks_var[2:], "months":months_var[2:]}

# Plotando os Dfs
fig_var_novas_inf_divul = go.Figure()
fig_var_novas_inf_divul = fig_var_novas_inf_divul.add_trace(go.Scatter(x = dfs["days"].index,
                               y = dfs["days"]["newCases"], 
                               name = "Variação Diária",
                               line = dict(color='orange')))

fig_var_novas_inf_divul = fig_var_novas_inf_divul.add_trace(go.Scatter(x = dfs["weeks"].index,
                               y = dfs["weeks"]["newCases"], 
                               name = "Variação Semanal",
                               line = dict(color='blue')))

fig_var_novas_inf_divul = fig_var_novas_inf_divul.add_trace(go.Scatter(x = dfs["months"].index,
                               y = dfs["months"]["newCases"], 
                               name = "Variação Mensal",
                               line = dict(color='red')))

fig_var_novas_inf_divul.update_layout(
                title='Variação de Novas Infecções Divulgadas',
                title_font_color= 'black',
                title_font_size = 20,
                title_x = 0.5)

fig_var_novas_inf_divul.show()

In [38]:
# Criando um dicionário para os Dfs
dict_cases = {"days" : days_var[2:], "weeks": weeks_var[2:], "months":months_var[2:]}

# Plotando os Dfs
fig_var_novos_obit_div = go.Figure()
fig_var_novos_obit_div = fig_var_novos_obit_div.add_trace(go.Scatter(x = dict_cases["days"].index,
                               y = dict_cases["days"]["newDeaths"], 
                               name = "Variação Diária",
                               line = dict(color='orange')))

fig_var_novos_obit_div = fig_var_novos_obit_div.add_trace(go.Scatter(x = dict_cases["weeks"].index,
                               y = dict_cases["weeks"]["newDeaths"], 
                               name = "Variação Semanal",
                               line = dict(color='blue')))

fig_var_novos_obit_div = fig_var_novos_obit_div.add_trace(go.Scatter(x = dict_cases["months"].index,
                               y = dict_cases["months"]["newDeaths"], 
                               name = "Variação Mensal",
                               line = dict(color='red')))

fig_var_novos_obit_div.update_layout(
                title='Variação de Novos Óbitos Divulgados',
                title_font_size=20,
                title_font_color='black',
                title_x =0.5)

fig_var_novos_obit_div.show()

### A forma que os dados são agregados faz muita diferença!
A comparação visual dos gráficos diário, semanal e mensal mostra o quão diferentes as informações obtidas podem ser, a depender da forma de agregação delas!

A análise da correlação diária não é suficiente, pois ela gera um problema: a ausência de transmissão da informação. Por exemplo, um aumento nos casos de covid só é percebido pela população alguns dias depois desse aumento ocorrer. Dessa forma, não existe um impacto imediato no mercado. Por isso, uma correlação diária de dados pode apresentar uma "misleading information" (informação enganosa), por não ser capaz de representar a realidade.


In [40]:
# Removendo os valores extremos das variações mensais e semanais:
out_dict = {
"Novas Infecções Mensais": months_var["newCases"],
"Novos Óbitos Mensais": months_var["newDeaths"],
"Novas Infecções Semanais": weeks_var["newCases"],
"Novos Óbitos Semanais": weeks_var["newDeaths"]
}

for item in out_dict:
    outliers = out_dict[item].between(out_dict[item].quantile(.05), out_dict[item].quantile(.9))
    outilers = pd.DataFrame(outliers)
    out_dict[item] = out_dict[item].multiply(outliers)

In [42]:
# Boxplot dos valores que restaram na base:
fig_boxplot = make_subplots(rows=1, cols=1)

for item in out_dict:
    fig_boxplot.add_trace(go.Box(y=out_dict[item], name = f"{item}"))

fig_boxplot.update_layout(height=600,
                            width=1000,
                            title_text="Box Plot das Variações",
                            title_font_size=20,
                            title_font_color='black',
                            title_x=0.5)
fig_boxplot.show()


# Comparação dos Casos Mensais de Covid com o IPCA

Base de dados disponível em:

<https://dadosabertos.bcb.gov.br/dataset/4449-indice-nacional-de-precos-ao-consumidor-amplo-ipca---precos-monitorados---total>

In [26]:
ipca = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.4449/dados?formato=csv")
ipca

<Response [200]>

In [27]:
ipca = ipca.content.decode("utf-8")
ipca = [x.split(';') for x in ipca.split('\r\n')]

In [28]:
# Removendo as aspas ("") prensentes em cada item do dataframe:
for row in range(0, len(ipca)):
    for item in range(0, len(ipca[row])):
        ipca[row][item] = ipca[row][item].replace('\"', "")

In [29]:
ipca_df = pd.DataFrame(ipca[1:], columns= ipca[0])

In [30]:
# Removendo a última linha, que possui valores nulos:
ipca_df = ipca_df[:-1]

In [31]:
# Colocando a data no formato datetime:
ipca_df["data"] = pd.to_datetime(ipca_df["data"], format="%d/%m/%Y")

In [32]:
# Trocando os pontos por vírgulas:
ipca_df["valor"] = [row.replace(",", ".") for row in ipca_df["valor"]]

In [33]:
# Colocando o valor no formato float:
ipca_df["valor"] = ipca_df["valor"].astype(float)

In [34]:
# Separando um dataframe para analizar o IPCA entre 2020 e os dias atuais:
ipca = ipca_df.loc[ipca_df["data"]>="2020-03-01"]

In [44]:
# Dicionário para o Df:
dict_ipca = {"ipca" : ipca[2:], "covid": months_var[2:]}

# plot the data
fig_ipca_x_covid = go.Figure()
fig_ipca_x_covid = fig_ipca_x_covid.add_trace(go.Scatter(x = dict_ipca["ipca"]["data"],
                               y = dict_ipca["ipca"]["valor"],
                               name = "IPCA no Brasil",
                               line = dict(color='blue')))

fig_ipca_x_covid = fig_ipca_x_covid.add_trace(go.Scatter(x = dict_ipca["covid"].index,
                               y = dict_ipca["covid"]["newCases"], 
                               name = "Número de Infecções",
                               line = dict(color='orange')))

fig_ipca_x_covid = fig_ipca_x_covid.add_trace(go.Scatter(x = dict_ipca["covid"].index,
                               y = dict_ipca["covid"]["newDeaths"], 
                               name = "Número de Óbitos",
                               line = dict(color='red')))

fig_ipca_x_covid.update_layout(
                            title='Variações Mensais: IPCA x Casos de Covid',
                            title_font_size=20,
                            title_font_color='black',
                            title_x=0.5)

fig_ipca_x_covid.show()

NameError: name 'ipca' is not defined

Aqui, voltamos a ter o problema da correlação. Como avaliar se o aumento em uma determinada data está relacionado com o aumento em uma data seguinte??

Um heatmap com a correlação entre o IPCA e os casos de covid, para um mesmo período, são apresentados na sessão seguinte.

# Comparação dos Casos de Covid com a Bolsa de Valores e o Dolar

Através da biblioteca yfinance

In [36]:
ibov_month = yf.download('^bvsp', start='2020-01-01', end="2022-11-01", interval='1mo')
usd_month = yf.download('USDBRL=X', start='2020-01-01', end="2022-11-01", interval='1mo')

ibov_week = yf.download('^bvsp', start='2020-01-01', end="2022-11-01", interval='1wk')
usd_week = yf.download('USDBRL=X', start='2020-01-01', end="2022-11-01", interval='1wk')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
# Obtendo as variações mensais e semanais:
ibov_mvar = ibov_month.pct_change()
usd_mvar = usd_month.pct_change()

ibov_wvar = ibov_week.pct_change()
usd_wvar = usd_week.pct_change()

In [38]:
# Criando o dicionário:
dict_ibov_m = {"ipca" : ipca[4:], "covid": months_var[2:], "ibov":ibov_mvar[4:], "usd":usd_mvar[4:]}

# Plotting:
fig_ipca_covid_ibov_usd = go.Figure()
fig_ipca_covid_ibov_usd = fig_ipca_covid_ibov_usd.add_trace(go.Scatter(x = dict_ibov_m["ipca"]["data"],
                               y = dict_ibov_m["ipca"]["valor"], 
                               name = "IPCA no Brasil",
                               line = dict(color='blue')))

fig_ipca_covid_ibov_usd = fig_ipca_covid_ibov_usd.add_trace(go.Scatter(x = dict_ibov_m["covid"].index,
                               y = dict_ibov_m["covid"]["newCases"], 
                               name = "Número de Infecções",
                               line = dict(color='orange')))

fig_ipca_covid_ibov_usd = fig_ipca_covid_ibov_usd.add_trace(go.Scatter(x = dict_ibov_m["covid"].index,
                               y = dict_ibov_m["covid"]["newDeaths"], 
                               name = "Número de Óbitos",
                               line = dict(color='red')))

fig_ipca_covid_ibov_usd = fig_ipca_covid_ibov_usd.add_trace(go.Scatter(x = dict_ibov_m["ibov"].index,
                               y = dict_ibov_m["ibov"]["Close"], 
                               name = "Variação na Bolsa de Valores",
                               line = dict(color='green')))

fig_ipca_covid_ibov_usd = fig_ipca_covid_ibov_usd.add_trace(go.Scatter(x = dict_ibov_m["usd"].index,
                               y = dict_ibov_m["usd"]["Close"], 
                               name = "Variação da Cotação do Dolar",
                               line = dict(color='purple')))


fig_ipca_covid_ibov_usd.update_layout(
                            title='Variações Mensais: IPCA, IBOV, USD e Casos de Covid',
                            title_font_size=20,
                            title_font_color='black',
                            title_x = 0.5)

fig_ipca_covid_ibov_usd.show()

In [45]:
# Criando o dicionário:
dict_ibov_w = {"covid": weeks_var[2:], "ibov":ibov_wvar[4:], "usd":usd_wvar[4:]}

# Plotting:
fig_ipca_covid_ibov_usd_week = go.Figure()

fig_ipca_covid_ibov_usd_week = fig_ipca_covid_ibov_usd_week.add_trace(go.Scatter(x = dict_ibov_m["covid"].index,
                               y = dict_ibov_m["covid"]["newCases"], 
                               name = "Número de Infecções"))

fig_ipca_covid_ibov_usd_week = fig_ipca_covid_ibov_usd_week.add_trace(go.Scatter(x = dict_ibov_m["covid"].index,
                               y = dict_ibov_m["covid"]["newDeaths"], 
                               name = "Número de Óbitos"))

fig_ipca_covid_ibov_usd_week = fig_ipca_covid_ibov_usd_week.add_trace(go.Scatter(x = dict_ibov_m["ibov"].index,
                               y = dict_ibov_m["ibov"]["Close"], 
                               name = "Variação na Bolsa de Valores"))

fig_ipca_covid_ibov_usd_week = fig_ipca_covid_ibov_usd_week.add_trace(go.Scatter(x = dict_ibov_m["usd"].index,
                               y = dict_ibov_m["usd"]["Close"], 
                               name = "Variação da Cotação do Dolar"))

fig_ipca_covid_ibov_usd_week.update_layout(
                                title='Variações Semanais: IBOV, USD e Casos de Covid',
                                title_font_size=20,
                                title_font_color='black',
                                title_x = 0.5)

fig_ipca_covid_ibov_usd_week.show()

NameError: name 'ibov_wvar' is not defined

Analisando o gráfico da cotação do dolar, em relação a bolsa brasileira, percebemos que um aumento na cotação do dolar resulta em um aumento na cotação da bolsa brasileira após uma ou duas semanas - experimente!

Por outro lado, a relação entre os casos de covid e a bolsa de valores é muito dificil de ser feita visualmente, sendo adequada a comparação por meio de outras ferramentas.

In [40]:
# Avaliando a correlação Mensal:
# Os números entre colchetes "[]" são para ajustar as datas do período:

month_var_df = pd.DataFrame({
    "ipca": [row for row in ipca["valor"]],
    "ipca_data": [row for row in ipca["data"]],
    
    "covid_newCases": [row for row in months_var["newCases"][:-2]],
    "covid_newDeaths": [row for row in months_var["newDeaths"][:-2]],
    "covid_data": [row for row in months_var.index[:-2]],
    
    "ibov": [row for row in ibov_mvar["Close"][2:-1]], 
    "usd": [row for row in usd_mvar["Close"][2:]] 
    
})
month_var_df.head()

,ipca,ipca_data,covid_newCases,covid_newDeaths,covid_data,ibov,usd
0,-0.22,2020-03-01,NaN,NaN,2020-02-29,-0.299044,0.161546
1,-2.06,2020-04-01,2910.000000,inf,2020-03-31,0.102520,0.027365
2,-1.02,2020-05-01,12.963758,27.603960,2020-04-30,0.085671,-0.000469
3,0.89,2020-06-01,4.276640,3.047248,2020-05-31,0.087560,0.013090
4,1.23,2020-07-01,1.089777,0.300663,2020-06-30,0.082646,-0.045945


In [41]:
# Avaliando a correlação Semanal:
# Os números entre colchetes "[]" são para ajustar as datas do período:

#week_var_df = pd.DataFrame({
#    "covid_newCases": [row for row in weeks_var["newCases"][:-3]], 
#    "covid_newDeaths": [row for row in weeks_var["newDeaths"][:-3]],
#    "ibov": [row for row in ibov_wvar["Close"][9:]],
#    "usd": [row for row in usd_wvar["Close"][9:]]
#    
#})
#week_var_df.head()

In [42]:
fig_corr_val_mensal = px.imshow(month_var_df.corr(), text_auto=True, title = "Correlação das Variações Mensais")
fig_corr_val_mensal.show()

C:\Users\Giga\AppData\Local\Temp\ipykernel_42800\2631872836.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [43]:
#fig_corr_val_week = px.imshow(week_var_df.corr(), text_auto=True ,title = "Correlação das Variações Semanais")
#fig_corr_val_week.show()

In [45]:
# Criando Dashboard com Storytelling
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H2(children='Covid no Brasil com impacto no IPCA (Índice Nacional de Preços ao Consumidor Amplo)',
    style={'textAlign': 'center' color:'#7FDBFF' }),
    html.H2("Análise realizada como trabalho em grupo do Cusro - DiversidadeTech - Let's Code by ADA",
    style={'textAlign': 'center'}),
    html.Div(children='Professor - Alex Lima.',
    style={'textAlign': 'center'}),
    html.Div(children='Alunos - Alysson, Gleilson Nivea, Renato e Tiago.',
    style={'textAlign': 'center'}),
    html.Div(children='----'*20,
    style={'textAlign': 'center'}),
    html.Div(children='No dia 17.11.2019 é confirmado o primeiro caso da CIVID-19 no mundo.',
    style={'textAlign': 'center'}),
    html.Div(children='Estudos do viroloigista Michael Worobey, identifica que se tratava de uma mulher',
    style={'textAlign': 'center'}),
    html.Div(children='vendedora de um mercado de animais, em Wuhan na China.',
    style={'textAlign': 'center'}),
    dcc.Link('https://dash.plotly.com/dash-core-components/link')
    
    html.Div(children='Inserir link.',
    style={'textAlign': 'center'}),


    
#option = ['Infectados Mês', 'Mortes Mês', 'Suspeitos Mês', 'Vacinado Mês', 'Completa Mês']
    dcc.Dropdown(['Infectados Mês', 'Mortes Mês', 'Suspeitos Mês', 'Vacinado Mês', 'Completa Mês'],
                    placeholder="Selecione o Gráfico",
                    searchable=False,
                    value='Completa Mês',
                    id = 'Lista de Gráficos'),
    dcc.Graph(
        id='graph_01',
        figure=fig_infec_susp_vacin_mortes
    )
    
])

@app.callback(
    Output('graph_01', 'figure'),
    Input('Lista de Gráficos', 'value')

)
def update_output(value):
    if value == 'Infectados Mês':
        fig_infec_susp_vacin_mortes = px.line(months, x=months.index, y="newCases",
               markers= True,
               title='Infecções Divulgadas ao Mês',
               labels = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["royalblue"])    
    elif value == 'Mortes Mês':
        fig_infec_susp_vacin_mortes = px.line(months, x=months.index, y="newDeaths",
               markers= True,
               title='Óbitos Divulgados ao Mês',
               labels = {
                "deathsMS": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["red"])
    elif value == 'Suspeitos Mês':
        fig_infec_susp_vacin_mortes = px.line(months, x=months.index, y="suspects",
                markers= True,
                title='Casos Suspeitos Divulgados por Mês',
                labels  = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["orange"])
    elif value == 'Vacinado Mês':
        fig_infec_susp_vacin_mortes = px.line(months, x=months.index, y="vaccinated",
                markers= True,
                title='Quantidade de Vacinados por Mês',
                labels  = {
                "newCases": "Quantidade",
                "date": "Data"
                },color_discrete_sequence=["green"])
    elif value == 'Completa Mês':
        fig_infec_susp_vacin_mortes = go.Figure()
        fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                                    y = scaled_months["newCases"],
                                    mode = 'lines+markers',
                                    name = "Infecções",
                                    line = dict(color='royalblue',width=2,dash='dot')))
        fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                                    y = scaled_months["suspects"],
                                    mode = 'lines+markers',
                                    name = "Suspeitos",
                                    line = dict(color='orange',width=1,dash='dot')))
        fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                                    y = scaled_months["vaccinated"],
                                    mode = 'lines+markers',
                                    name = "Vacinados"))
        fig_infec_susp_vacin_mortes = fig_infec_susp_vacin_mortes.add_trace(go.Scatter(x = months.index, 
                                    y = scaled_months["newDeaths"],
                                    mode = 'lines+markers',
                                    name = "Óbitos",
                                    line = dict(color='red')
                                    ))
        fig_infec_susp_vacin_mortes.update_layout(
                                        title='Comparando Novos casos, suspeitos, vacinados e Óbitos Mês',
                                        title_x = 0.5)

        fig_infec_susp_vacin_mortes #f' Você Selecionou -> {value}'
    return fig_infec_susp_vacin_mortes


if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [15/Nov/2022 19:08:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:12] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2022 19:08:24] "POST /_dash-update-component HTTP/1.1" 200